In [4]:
import pandas as pd

matches_df = pd.read_csv("datasets/WorldCupMatches.csv")
players_df = pd.read_csv("datasets/WorldCupPlayers.csv")
worldcups_df = pd.read_csv("datasets/WorldCups.csv")

print("=== WorldCupMatches Columns ===")
print(matches_df.columns)

print("=== WorldCupPlayers Columns ===")
print(players_df.columns)

print("=== WorldCups Columns ===")
print(worldcups_df.columns)

=== WorldCupMatches Columns ===
Index(['Year', 'Datetime', 'Stage', 'Stadium', 'City', 'Home Team Name',
       'Home Team Goals', 'Away Team Goals', 'Away Team Name',
       'Win conditions', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2',
       'RoundID', 'MatchID', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')
=== WorldCupPlayers Columns ===
Index(['RoundID', 'MatchID', 'Team Initials', 'Coach Name', 'Line-up',
       'Shirt Number', 'Player Name', 'Position', 'Event'],
      dtype='object')
=== WorldCups Columns ===
Index(['Year', 'Country', 'Winner', 'Runners-Up', 'Third', 'Fourth',
       'GoalsScored', 'QualifiedTeams', 'MatchesPlayed', 'Attendance'],
      dtype='object')
